<div align="center"><b><h1> Buscando dados de vídeos do YouTube - Parte 2 </h1></b></div>

<img src="youtube.com_.jpg" width="480">

### Objetivo
Apresentar um exemplo de como baixar dados de vídeos de um canal a partir da API do YouTube. Neste caso, foi utilizado o canal de humor "Porta dos Fundos"

Para importar nossa base de dados vamos utilizar a seguinte estratégia:<br>
1 - utilizar a biblioteca "yt_videos_list" para retornar a lista de vídeos publicados no canal desejado. Essa biblioteca facilita bastante este processo;<br>
2 - em seguida, será realizada a conexão com a API do YouTube e um loop através dos vídeos para obter informações como título, visualizações, elenco, diretor(a), roteiristas, likes, dislikes e data de publicação as quais serão armazenadas em um dicionário;<br>
3 - após isto, os dados serão carregados em um pandas dataframe e se procederá às devidas limpezas para se obter a base de dados final, pronta para análise.
<br>
<br>
Vamos lá!!!

### Importando as libraries necessárias

In [15]:
from yt_videos_list import ListCreator
import pandas as pd
import re
import time
from googleapiclient.discovery import build
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import HTML
InteractiveShell.ast_node_interactivity = "all"

### Instanciando o driver do navegador e o listcreator
As opções para o driver são as seguintes:<br>
Firefox (default) (driver='firefox') <br>
Opera(driver='opera') <br>
Safari (MacOS only) (driver='safari') <br>
Chrome (driver='chrome') <br>
Brave (driver='brave') <br>
Edge (Windows only) (driver='edge') <br>

In [3]:
my_driver = 'chrome' # substitua pelo driver (nome) do seu navegador, conforme acima
lc = ListCreator(csv=True,
                 md=False,
                 txt=False,
                 #headless=True,
                 driver=my_driver,
                 scroll_pause_time=2,
                 reverse_chronological=True)

### Criando a lista de vídeos do canal

In [4]:
filename = 'PortadosFundosvideo'
lc.create_list_for(url='https://www.youtube.com/c/PortadosFundos/videos', log_silently=True, file_name=filename)

'PortadosFundosvideo'

### Carregando a lista de vídeos em um dataframe e observando os primeiros registros

In [10]:
df = pd.read_csv(filename+'.csv')
df.head()

,Video Number,Video Title,Video URL,Watched?,Watch again later?,Notes
0,1651,TÁ TUDO BEM,https://www.youtube.com/watch?v=xsLrkt3XEaM,NaN,NaN,NaN
1,1650,KATIA,https://www.youtube.com/watch?v=onNxHMHBdoA,NaN,NaN,NaN
2,1649,RIR É UM ATO DE RESISTÊNCIA,https://www.youtube.com/watch?v=5wAZaD2Z4IY,NaN,NaN,NaN
3,1648,SONHEI COM VOCÊ,https://www.youtube.com/watch?v=LVwbiEtAJ6Q,NaN,NaN,NaN
4,1647,DIVERSIDADE,https://www.youtube.com/watch?v=o84iIil6DU0,NaN,NaN,NaN


### Excluindo os vídeos de making of para reduzir dupla contagem de atores/diretores/roteiristas na mesma peça

In [11]:
df = df[~df['Video Title'].str.contains('MAKING OF')]
df.shape

(1410, 6)

### Gerando uma lista de ID's dos vídeos, para consulta das informações via API

In [12]:
df['video_id'] = df['Video URL'].str.extract(r'v=(.*)', expand = True)
video_id_list = df['video_id'].tolist()

### Verificando quantidade de vídeos e observando os 10 primeiros registros

In [13]:
len(video_id_list)
video_id_list[:10]

1410

['xsLrkt3XEaM',
 'onNxHMHBdoA',
 '5wAZaD2Z4IY',
 'LVwbiEtAJ6Q',
 'o84iIil6DU0',
 'J7M-9MJThu4',
 'dmlh-1ettn0',
 '2toRdsfyG2w',
 'RhcN-5RmVBM',
 'rd5YldXpG-g']

### Conectando à API do Google para retornar as informações desejadas
Caso não saiba como gerar uma API Key para o YouTube, é só dar uma conferida no vídeo do link abaixo: <br>

https://youtu.be/3jZ5vnv-LZc

In [35]:
youTubeApiKey= # Aqui você deve inserir sua API Key, gerada via Google Cloud Plataform
youtube=build('youtube','v3', developerKey=youTubeApiKey)

### Buscando os dados desejados na API do YouTube, para os vídeos na lista

In [ ]:
%%time

# incializando o dicionário que irá guardar as informações extraídas dos vídeos
result = {"title":[], "views":[], "cast":[], "script":[], "directing":[], "date_published":[], "likes":[], "dislikes":[]}

n = 0

for video_id in video_id_list:
    print(n)
    print(video_id)    
    stat = youtube.videos().list(id=video_id, part='statistics').execute()
    snip = youtube.videos().list(id=video_id, part='snippet').execute()
    description=str(snip['items'][0]['snippet']['localized'])
    
    for character in ['Elenco', 'Elenco:', 'elenco', 'ELENCO:', "ELENCO "]:
        description = description.replace(character, "ELENCO")

    for character in ['Roteiro', 'Roteiro:', 'roteiro', 'ROTEIRO:', "ROTEIRO "]:
        description = description.replace(character, "ROTEIRO")

    for character in ['Direção', 'Direção:', 'direção', 'DIREÇÃO:', "DIREÇÃO "]:
        description = description.replace(character, "DIREÇÃO")

    for character in [' \\n ', ' \\n', '\\n ']:
        description = description.replace(character, "\\n")
    
    #title
    result["title"].append(snip['items'][0]['snippet']['title'])
    #views
    try:
        result["views"].append(stat['items'][0]['statistics']['viewCount'])
    except:
        result["views"].append('')
    #cast
    try:
        result["cast"].append(''.join(re.findall(r'\\nELENCO\\n(.*?)\\n\\n', description)))
    except:
        result["cast"].append('')
    #script
    try:        
        result["script"].append(''.join(re.findall(r'\\nROTEIRO\\n(.*?)\\n\\n', description)))
    except:
        result["script"].append('')
    #directing
    try:
        result["directing"].append(''.join(re.findall(r'\\nDIREÇÃO\\n(.*?)\\n\\n', description)))    
    except:
        result["directing"].append('')
    # date published
    result["date_published"].append(snip['items'][0]['snippet']['publishedAt'])
    #likes
    try:
        result["likes"].append(stat['items'][0]['statistics']['likeCount'])
    except:
        result["likes"].append('')
    #dislikes
    try:
        result["dislikes"].append(stat['items'][0]['statistics']['dislikeCount'])
    except:
        result["dislikes"].append('')
    n = n + 1
    time.sleep(1)

### Carregando os dados em um dataframe para realizar as limpezas necessárias

In [60]:
df = pd.DataFrame.from_dict(result)
df.shape
df.head(10)
df.info()

(1410, 8)

,title,views,cast,script,directing,date_published,likes,dislikes
0,TÁ TUDO BEM,334867,Fábio de Luca\nRafael Infante,Gabriel Esteves,Zaga Martelletto,2021-05-10T14:00:32Z,44800,651
1,KATIA,384456,Evelyn Castro\nFábio de Luca,Matheus MAD,Bianca Frossard,2021-05-08T14:00:11Z,37581,511
2,RIR É UM ATO DE RESISTÊNCIA,412737,,,,2021-05-06T14:00:04Z,92594,688
3,SONHEI COM VOCÊ,423480,Estevam Nabote\nNathalia Cruz,Matheus MAD,Bianca Frossard,2021-05-03T14:00:07Z,46417,338
4,DIVERSIDADE,477663,Gregório Duviver\nRafael Infante,Gustavo Vilela,Rodrigo Van Der Put,2021-05-01T14:00:31Z,53420,799
5,VIDA EM MARTE,543461,João Vicente de Castro\nPedro Benevides,Matheus MAD,Gigi Soares,2021-04-29T14:00:07Z,51774,1008
6,LEILÃO ONLINE,869218,Fábio Porchat,Jhonathan Marques\nEdu Araujo,Bianca Frossard,2021-04-26T14:00:14Z,72699,843
7,"AMIGA, PARE",467961,Thati Lopes\nEvelyn Castro,Nathalia Cruz,Rodrigo Van Der Put,2021-04-24T14:00:01Z,42214,1171
8,ERIC QUERIDO,426112,Gregório Duvivier\nEvelyn Castro\nPedro Benevi...,Gregório Duvivier,Rodrigo Van Der Put,2021-04-22T14:00:09Z,42127,1379
9,CÂMERA LIGADA,633554,Rafael Infante\nNoemia Oliveira\nEstevam Nabot...,Jhonatan Marques,Bianca Frossard,2021-04-19T14:00:14Z,53421,522


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 0 to 1409
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           1410 non-null   object
 1   views           1410 non-null   object
 2   cast            1410 non-null   object
 3   script          1410 non-null   object
 4   directing       1410 non-null   object
 5   date_published  1410 non-null   object
 6   likes           1410 non-null   object
 7   dislikes        1410 non-null   object
dtypes: object(8)
memory usage: 88.2+ KB


### Convertendo as variáveis 'views', 'likes' e 'dislikes' de string para numérico

In [61]:
df['views'] = df.views.astype(float)
df['likes'] = df.likes.replace('', np.nan).astype(float)
df['dislikes'] = df.dislikes.replace('', np.nan).astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 0 to 1409
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1410 non-null   object 
 1   views           1410 non-null   float64
 2   cast            1410 non-null   object 
 3   script          1410 non-null   object 
 4   directing       1410 non-null   object 
 5   date_published  1410 non-null   object 
 6   likes           1402 non-null   float64
 7   dislikes        1402 non-null   float64
dtypes: float64(3), object(5)
memory usage: 88.2+ KB


### Substituindo o '\\\n' por vírgula para facilitar dividir atores, roteiristas e diretores em linhas

In [64]:
df['cast'] = df['cast'].replace(r'\\n',',', regex=True)
df['cast'] = df['cast'].replace('-',',', regex=True)
df['script'] = df['script'].replace(r'\\n',',', regex=True)
df['directing'] = df['directing'].replace(r'\\n',',', regex=True)

In [65]:
df.tail(50)

,title,views,cast,script,directing,date_published,likes,dislikes
1360,TAXISTA,7479297.0,"Fábio Porchat,Luis Lobianco",,,2012-12-20T12:59:20Z,103108.0,3088.0
1361,CLARICE FALCÃO - OITAVO ANDAR,16431313.0,"Mulher: Clarice Falcão,Homem: Rodrigo Magal,Do...",,,2012-12-17T16:50:53Z,275821.0,5362.0
1362,COM QUEM SERÁ?,13043259.0,"Clarice Falcão,Fábio Porchat,Gabriel Totoro",,,2012-12-13T12:58:03Z,233097.0,3116.0
1363,DR,3636586.0,Julia Rabello,,,2012-12-10T12:57:41Z,53210.0,1629.0
1364,TROCADO,6322735.0,"Julia Rabello,Rafael Infante",,,2012-12-06T12:58:26Z,92404.0,1736.0
1365,BARATA NO BANHEIRO,10486388.0,"Antonio Tabet,Clarice Falcão,Gregorio Duvivier...",,,2012-12-03T13:00:28Z,157403.0,5723.0
1366,"TEQUILA - ""Quem tá dentro?""",3664137.0,"Fábio Porchat,Gabriel Totoro,Letícia Lima,Luis...",,,2012-11-29T12:59:57Z,33897.0,2476.0
1367,TROCA DE PRESENTE,3416367.0,"Clarice Falcão,Julia Rabello",,,2012-11-26T12:58:44Z,33536.0,1105.0
1368,VAN,7338839.0,"Fábio Porchat,Gregorio Duvivier",,,2012-11-22T13:01:39Z,99500.0,2128.0
1369,TIPO...,5934151.0,"Julia Rabello,Marcus Majella",,,2012-11-19T12:57:48Z,63705.0,1352.0


### Dividindo atores, roteiristas e diretores em linhas

In [66]:
df_exploded = df.assign(cast=df.cast.str.split(",")).explode('cast')
df_exploded2 = df_exploded.assign(script=df_exploded.script.str.split(",")).explode('script')
df_exploded3 = df_exploded2.assign(directing=df_exploded2.directing.str.split(",")).explode('directing')
df_exploded3.shape
df_exploded3.tail(10)

(4369, 8)

,title,views,cast,script,directing,date_published,likes,dislikes
1406,CSI NOVA IGUAÇU #8,1183493.0,Antonio Tabet,Antonio Tabet,Ian SBF,2012-08-08T00:54:43Z,NaN,NaN
1406,CSI NOVA IGUAÇU #8,1183493.0,Gabriel Totoro,Antonio Tabet,Ian SBF,2012-08-08T00:54:43Z,NaN,NaN
1406,CSI NOVA IGUAÇU #8,1183493.0,Gustavo Chagas,Antonio Tabet,Ian SBF,2012-08-08T00:54:43Z,NaN,NaN
1406,CSI NOVA IGUAÇU #8,1183493.0,Letícia Lima,Antonio Tabet,Ian SBF,2012-08-08T00:54:43Z,NaN,NaN
1407,COCAÍNA - Porta Dos Fundos Nº 1,2324249.0,Julia Rabello,,,2012-08-08T00:54:15Z,NaN,NaN
1407,COCAÍNA - Porta Dos Fundos Nº 1,2324249.0,Antonio Tabet,,,2012-08-08T00:54:15Z,NaN,NaN
1408,TRAVECO DA FIRMA - Porta dos Fundos Nº 1,12185884.0,Fabio Porchat,,,2012-08-08T00:53:43Z,NaN,NaN
1408,TRAVECO DA FIRMA - Porta dos Fundos Nº 1,12185884.0,Gustavo Chagas,,,2012-08-08T00:53:43Z,NaN,NaN
1408,TRAVECO DA FIRMA - Porta dos Fundos Nº 1,12185884.0,Rafael Infante,,,2012-08-08T00:53:43Z,NaN,NaN
1409,PORTA DOS FUNDOS N°1,8040264.0,,,,2012-08-06T14:20:18Z,NaN,NaN


### Convertendo o campo "date_published" de string para datetime

In [72]:
df_exploded3['dt_published'] = pd.to_datetime(df_exploded3['date_published'].str.slice(start=0, stop=10), format='%Y-%m-%d')
df_exploded3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4369 entries, 0 to 1409
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           4369 non-null   object        
 1   views           4369 non-null   float64       
 2   cast            4369 non-null   object        
 3   script          4369 non-null   object        
 4   directing       4369 non-null   object        
 5   date_published  4369 non-null   object        
 6   likes           4348 non-null   float64       
 7   dislikes        4348 non-null   float64       
 8   dt_published    4369 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 341.3+ KB


### Convertendo todas as observações das variáveis string para minúsculo
Isso é necessário para evitar que o python trate "Fábio de Luca" como sendo diferente de "Fábio de luca"

In [73]:
df_exploded4 = df_exploded3.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)
df_exploded4.head()

,title,views,cast,script,directing,date_published,likes,dislikes,dt_published
0,tá tudo bem,334867.0,fábio de luca,gabriel esteves,zaga martelletto,2021-05-10t14:00:32z,44800.0,651.0,2021-05-10
0,tá tudo bem,334867.0,rafael infante,gabriel esteves,zaga martelletto,2021-05-10t14:00:32z,44800.0,651.0,2021-05-10
1,katia,384456.0,evelyn castro,matheus mad,bianca frossard,2021-05-08t14:00:11z,37581.0,511.0,2021-05-08
1,katia,384456.0,fábio de luca,matheus mad,bianca frossard,2021-05-08t14:00:11z,37581.0,511.0,2021-05-08
2,rir é um ato de resistência,412737.0,,,,2021-05-06t14:00:04z,92594.0,688.0,2021-05-06


### Dropando o campo "date_published" e salvando o dataframe em csv para utilização posterior

In [74]:
df_final = df_exploded4.drop(['date_published'], axis=1)
df_final.shape
df_final.head()

(4369, 8)

,title,views,cast,script,directing,likes,dislikes,dt_published
0,tá tudo bem,334867.0,fábio de luca,gabriel esteves,zaga martelletto,44800.0,651.0,2021-05-10
0,tá tudo bem,334867.0,rafael infante,gabriel esteves,zaga martelletto,44800.0,651.0,2021-05-10
1,katia,384456.0,evelyn castro,matheus mad,bianca frossard,37581.0,511.0,2021-05-08
1,katia,384456.0,fábio de luca,matheus mad,bianca frossard,37581.0,511.0,2021-05-08
2,rir é um ato de resistência,412737.0,,,,92594.0,688.0,2021-05-06


In [75]:
df_final.to_csv('porta_dos_fundos_shortway_cleaned.csv', index=False)